# **P1 Consultas médicas especializadas de Porto Alegre - RS**

## **Projeto**

**AUTOR:** José Henrique de Oliveira<br>
**LINKEDIN:** [jholiveira94](https://www.linkedin.com/in/jholiveira94/)<br>
**GITHUB:** [JHOliveira94](https://github.com/JHOliveira94)

### **Objetivo**
Com este projeto, tem-se como objetivo elaborar e apresentar operações de CRUD (Create, Read, Update, Delete) com Python em banco de dados MySQL.\
Para tal, algumas etapas serão necessárias e, com elas em mente, assim o projeto será dividido:
- **P1:** ETL (Extract, Transform, Load)
    - Nessa etapa, os dados serão carregados, tradados e inseridos em um banco de dados MySQL usando Python.
- **P2:** CRUD (Create, Read, Update, Delete)
    - Nessa etapa, serão realizadas as operações de CRUD no banco de dados MySQL usando Python.
    - **Documento:** P2_SaudePOA_CRUD.ipynb

## **Sobre o conjunto de dados**

**DADOS ORIGINAIS:**
- https://dadosabertos.poa.br/dataset/gercon-gerenciamento-de-consultas

**ARQUIVOS:**
- Solicitação de consultas especializadas 2024 (.csv)
- Dicionário de Dados (.pdf)

**ACESSO EM:** 24/01/2025

Segundo o **Dicionário de Dados**:

**Identificação e Datas**\
	IDSOLICITACAO → Identificação única da solicitação.\
	DATAHORA_SOLICITACAO → Data e hora da solicitação.\
	DATAHORA_CONSULTA → Data e hora do agendamento da consulta (se houver).

**Status da Solicitação**\
	SITUACAOSOLICITACAO → Situação da solicitação (ex.: Solicitada, Cancelada, Realizada).\
	SITUACAOAGENDA → Status da agenda (ex.: Agendada, Confirmada, Cancelada).

**Informações sobre Regulação e Prioridade**\
	CORREGULADOR → Prioridade atribuída pelo regulador (Vermelho, Laranja, Amarelo, Verde, Azul).\
	STATUSREGIONALIZAÇÃO → Indica se há regionalização cadastrada para a especialidade e município.\
	CENTRALREGULACAO → Central de regulação responsável pela solicitação.\
	CENTRALREGULACAOORIGEM → Central de regulação original da solicitação.

**Tempo de Espera e Atendimento**\
	DIASFILAESPERA → Dias de espera entre solicitação e agendamento.

**Informações sobre a Especialidade**\
	ESPECIALIDADE → Especialidade médica específica (ex.: Cardiologia Pediátrica, Cirurgia Plástica).\
	ESPECIALIDADEMAE → Categoria geral da especialidade (ex.: Cardiologia, Cirurgia).

**Informações do Paciente**\
	IDPACIENTE → Identificação do paciente.\
	MUNICIPIO_RESIDENCIA → Município de residência do paciente.\
	IDADE → Idade do paciente.\
	CID → Código da Classificação Internacional de Doenças (CID-10).

**Unidade Solicitante**\
	CNES_SOLICITANTE → Código do estabelecimento solicitante no Cadastro Nacional de Estabelecimentos de Saúde (CNES).\
	UNIDADESOLICITANTE → Nome da unidade solicitante.\
	TIPOUNIDADE → Tipo da unidade solicitante (ex.: Hospital, UBS).\
	MUNICIPIOSOLICITANTE → Município da unidade solicitante.\
	COORDENADORIASOLICITANTE → Coordenadoria de saúde à qual a unidade solicitante está vinculada.\
	REGIAOSAUDESOLICITANTE → Região de saúde da unidade solicitante.

**Unidade Executante (se houver agendamento)**\
	CNESEXECUTANTE → Código do estabelecimento executante no CNES.\
	UNIDADEEXECUTANTE → Nome da unidade executante.\
	MUNICIPIO_EXECUTANTE → Município da unidade executante.

**Outras Informações**\
	COMPLEXIDADE → Classificação do atendimento (Alta ou Média Complexidade).\
	TIPOCONSULTA → Indica se é primeira consulta ou retorno.\
	POSSUIUNIDADEINDICADA → Indica se a solicitação tem uma unidade específica designada para atendimento.

## **1. Preparações**

In [33]:
# Para processo de ETL
import pandas as pd
import numpy as np

# Para tratamento de caracteres latinos, como "Ã" e "Ç"
import unicodedata 

# Para conexão com banco de dados MYSQL
%pip install sqlalchemy mysql-connector-python
from sqlalchemy import create_engine
import mysql.connector

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
def remove_acentos(texto):
    """
    Remove acentos e caracteres latinos de uma string.
    
    Parâmetro:
        texto: str
        - Texto que será analisado e em que os caracteres latinos serão substituídos.
    
    Retorna:
        str
        - Valor tratado com caracteres especiais substituídos.
    """
    try:
        texto = unicode(texto, 'utf-8')
    except NameError:
        pass
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    return str(texto)

## **2. Extract:** Leitura do conjunto de dados
Em testes iniciais, houve erro ao abir o documento csv em função do encoder.\
Assim, faz-se necessário testar alguns encoders mais comuns com finalidade de carregar os dados de forma adequada.

In [35]:
# Lista de codificações comuns para teste
codificacoes = ['utf-8', 'ISO-8859-1', 'latin1', 'cp1252'] 

for codificacao in codificacoes:
    try:
        df = pd.read_csv('gercon_solicitacoes_2024.csv', encoding=codificacao, sep = ';', low_memory=False)
        print(f"Arquivo lido com codificação: {codificacao}")
        break  # Sai do loop se a leitura for bem-sucedida
    except UnicodeDecodeError:
        print(f"Erro ao tentar ler com codificação: {codificacao}")
    except Exception as e:
        print(f"Erro desconhecido: {e}")

# Confirmação da criação do DataFrame
if 'df' in locals():
    print(" -> DataFrame criado com sucesso!")
else:
    print("Nenhuma codificação funcionou. Tente outras codificações ou leia o arquivo em um editor de texto.")

Erro ao tentar ler com codificação: utf-8
Arquivo lido com codificação: ISO-8859-1
 -> DataFrame criado com sucesso!


In [36]:
# Conferência e visão geral do conjunto de dados

df.head(2)

,data_extracao,idsolicitacao,situacaosolicitacao,data_solicitacao,corregulador,diasfilaespera,complexidade,especialidade,especialidademae,cid,...,cnesexecutante,unidadeexecutante,municipio_executante,statusregionalizacao,centralregulacao,centralregulacaoorigem,tipoconsulta,coordenadoriasolicitante,regiaosaudesolicitante,possuiunidadeindicada
0,2025-01-07 00:00:00.000,5075360,REALIZADA,2024-01-05 00:00:00.000,LARANJA,12,MEDIA,OSTEOPATIA GERAL,OSTEOPATIA,M461,...,2237385.0,AMBULATORIO DE ESPECIALIDADES IAPI,PORTO ALEGRE,NaN,REGULACAO AMBULATORIAL,REGULACAO AMBULATORIAL,PRIMEIRA,1 CRS,REGIÃO DE SAÚDE 10,NÃO
1,2025-01-07 00:00:00.000,5372229,REALIZADA,2024-03-14 00:00:00.000,LARANJA,1,MEDIA,OSTEOPATIA GERAL,OSTEOPATIA,M797,...,2237385.0,AMBULATORIO DE ESPECIALIDADES IAPI,PORTO ALEGRE,NaN,REGULACAO AMBULATORIAL,REGULACAO AMBULATORIAL,PRIMEIRA,1 CRS,REGIÃO DE SAÚDE 10,NÃO


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 29 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   data_extracao             1048575 non-null  object 
 1   idsolicitacao             1048575 non-null  int64  
 2   situacaosolicitacao       1048575 non-null  object 
 3   data_solicitacao          1048575 non-null  object 
 4   corregulador              1048575 non-null  object 
 5   diasfilaespera            1048575 non-null  int64  
 6   complexidade              1048575 non-null  object 
 7   especialidade             1048575 non-null  object 
 8   especialidademae          1048575 non-null  object 
 9   cid                       1048575 non-null  object 
 10  idpaciente                1048575 non-null  int64  
 11  municipio_residencia      1048575 non-null  object 
 12  idade                     1048575 non-null  int64  
 13  cnes_solicitante          1

Com os dados carregados com sucesso, aponta-se:
- O número de linhas e colunas correspondem com o arquivo.
- Há colunas que podem ser descartadas?
- As colunas **data_extracao**, **data_solicitacao** e **datahora_consulta** necessitam de tratamento de tipagem, pois **são do tipo data**, mas estão sendo lidas como object.
- Necessidade de estudar a presença de valores nulos.

## **3. Transform:** Adequação dos dados

### **3.1. Limpeza**
Visando diminuir a carga de dados, as colunas serão analisadas em prol de definir se alguma pode ser descartada.

In [38]:
# Quantos valores únicos cada coluna apresenta?

df.nunique()

data_extracao                     1
idsolicitacao               1048575
situacaosolicitacao              16
data_solicitacao                366
corregulador                      6
diasfilaespera                  373
complexidade                      2
especialidade                   276
especialidademae                 66
cid                            8450
idpaciente                   827219
municipio_residencia            613
idade                           113
cnes_solicitante               3214
unidadesolicitante             3061
tipounidade                      26
municipiosolicitante            497
situacaoagenda                    5
datahora_consulta               379
cnesexecutante                  444
unidadeexecutante               417
municipio_executante            167
statusregionalizacao              3
centralregulacao                 23
centralregulacaoorigem           23
tipoconsulta                      1
coordenadoriasolicitante         18
regiaosaudesolicitante      

Ao analisar quantos valores únicos cada coluna apresenta, destaca-se:
- **data_extracao**, **complexidade**, **tipoconsulta** e **possuiunidadeindicada** apresentam pouquíssima variedade de valores, portanto serão analisadas melhor.
- **idsolicitacao** apresenta quantidade de valores únicos igual ao número de registros, o que pode ser utilizado como índice em consultas futuras, logo, vale mantê-la.

In [39]:
# Presença de dados nulos nas colunas em destaque

df[['data_extracao', 'complexidade', 'tipoconsulta', 'possuiunidadeindicada']].isnull().sum()

data_extracao                 0
complexidade                  0
tipoconsulta             471913
possuiunidadeindicada         0
dtype: int64

In [40]:
# Composição dos valores únicos de cada coluna

for i in ['data_extracao', 'complexidade', 'tipoconsulta', 'possuiunidadeindicada']:
    print(f'{i}: ', df[i].unique())

data_extracao:  ['2025-01-07 00:00:00.000']
complexidade:  ['MEDIA' 'ALTA']
tipoconsulta:  ['PRIMEIRA' nan]
possuiunidadeindicada:  ['NÃO' 'SIM']


Com essas análises, conclui-se:
- **Descartar:** A coluna **data_extracao** apresenta valor idêntico para todos os registros, o que não auxilia em quaisquer estudos ou diferenciações como um ID.
- As colunas **complexidade**, e **possuiunidadeindicada** apresentam poucos valores únicos, porém não apresentam valores nulos, portanto podem ser úteis em futuros estudos, logo serão mantidas.
- A coluna **tipoconsulta** apresenta alta quantidade de valores nulos e apenas um valor único. Por hora será mantida, porém análises serão necessárias para compreender seus valores nulos. 

In [41]:
# Garantindo integridade dos dados originais
df_original = df.copy()

# Descartando a coluna data_extracao
df.drop(columns = 'data_extracao', inplace = True)

In [42]:
# Confirmação da alteração

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 28 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   idsolicitacao             1048575 non-null  int64  
 1   situacaosolicitacao       1048575 non-null  object 
 2   data_solicitacao          1048575 non-null  object 
 3   corregulador              1048575 non-null  object 
 4   diasfilaespera            1048575 non-null  int64  
 5   complexidade              1048575 non-null  object 
 6   especialidade             1048575 non-null  object 
 7   especialidademae          1048575 non-null  object 
 8   cid                       1048575 non-null  object 
 9   idpaciente                1048575 non-null  int64  
 10  municipio_residencia      1048575 non-null  object 
 11  idade                     1048575 non-null  int64  
 12  cnes_solicitante          1048575 non-null  int64  
 13  unidadesolicitante        1

### **3.2. Tipagem dos dados**
Como visto anteriormente, as colunas referentes à datas necessitam revisão de suas tipagens.

In [43]:
# Revisando tipagem das colunas de datas

df[['data_solicitacao', 'datahora_consulta']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   data_solicitacao   1048575 non-null  object
 1   datahora_consulta  576662 non-null   object
dtypes: object(2)
memory usage: 16.0+ MB


In [44]:
# Verificação do formato

df[['data_solicitacao', 'datahora_consulta']].head(3)

,data_solicitacao,datahora_consulta
0,2024-01-05 00:00:00.000,2024-01-31 00:00:00.000
1,2024-03-14 00:00:00.000,2024-03-19 00:00:00.000
2,2024-08-14 00:00:00.000,2024-08-27 00:00:00.000


In [45]:
# As horas apresentadas nas datas aparentam ser iguais a zero
# Verificação das horas nas datas

df_solicitacao = df[['data_solicitacao']].copy()
df_solicitacao['horas'] = df_solicitacao['data_solicitacao'].str[-12:]

df_consulta = df[['datahora_consulta']].copy()
df_consulta['horas'] = df_consulta['datahora_consulta'].str[-12:]

print(df_solicitacao['horas'].unique())
print(df_consulta['horas'].unique())

['00:00:00.000']
['00:00:00.000' nan]


In [46]:
df_consulta

,datahora_consulta,horas
0,2024-01-31 00:00:00.000,00:00:00.000
1,2024-03-19 00:00:00.000,00:00:00.000
2,2024-08-27 00:00:00.000,00:00:00.000
3,2024-10-08 00:00:00.000,00:00:00.000
4,2024-10-15 00:00:00.000,00:00:00.000
...,...,...
1048570,NaN,NaN
1048571,NaN,NaN
1048572,NaN,NaN
1048573,NaN,NaN


In [47]:
df_consulta.isnull().sum()

datahora_consulta    471913
horas                471913
dtype: int64

Como pode-se ver, as horas em ambas as colunas são idênticas a zero, logo podem ser descartadas.\
A coluna **datahora_consulta** apresenta valores nulos, porém, como pode-se consultar no dicionário dos dados, há consultas sem agendamento confirmado, o que explica os nulos aqui.

In [48]:
# Adequando a tipagem no DataFrame trabalhado

for i in ['data_solicitacao', 'datahora_consulta']:
    df[i] = pd.to_datetime(df[i])

In [49]:
# Confirmação das alterações

df[['data_solicitacao', 'datahora_consulta']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   data_solicitacao   1048575 non-null  datetime64[ns]
 1   datahora_consulta  576662 non-null   datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 16.0 MB


In [50]:
df[['data_solicitacao', 'datahora_consulta']].head(5)

,data_solicitacao,datahora_consulta
0,2024-01-05,2024-01-31
1,2024-03-14,2024-03-19
2,2024-08-14,2024-08-27
3,2024-09-09,2024-10-08
4,2024-10-02,2024-10-15


Além das colunas de data, as colunas relacionadas a códigos de identificação que estão com tipagem numérica podem ser adequadas para string, visto que podem interferir em análises numéricas.

In [51]:
# Adequação da tipagem

for i in ['idsolicitacao', 'idpaciente', 'cnes_solicitante', 'cnesexecutante']:
    df[i] = df[i].astype(str)

In [52]:
# Confirmação da mudança

df[['idsolicitacao', 'idpaciente', 'cnes_solicitante', 'cnesexecutante']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   idsolicitacao     1048575 non-null  object
 1   idpaciente        1048575 non-null  object
 2   cnes_solicitante  1048575 non-null  object
 3   cnesexecutante    1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


Com essas etapas concluídas:
- As tipagens estão adequadas aos dados.
- As horas em cada coluna estão suprimidas pelo próprio python e assim podem ficar, como visto antes.

### **3.3. Caracteres especiais**
Como visto no carregamento dos dados, o encoder usado é o XXX em função da presença de caracteres especiais como "Ã" e "Ç".\
Para garantir que os dados sejam lidos posteriormente em outros softwares, será feito o tratamento desses caracteres usando a biblioteca **unicode**.

In [53]:
# Exemplo de caracteres especiais no DataFrame

df[['regiaosaudesolicitante']].head(2)

,regiaosaudesolicitante
0,REGIÃO DE SAÚDE 10
1,REGIÃO DE SAÚDE 10


In [54]:
# Substituição dos caracteres especias em colunas do tipo object

for coluna in df.columns:
    if df[coluna].dtype == "object":
        df[coluna] = df[coluna].astype(str).apply(remove_acentos)

In [55]:
# Amostra do tratamento dos caracteres especiais

df[['regiaosaudesolicitante']].head(2)

,regiaosaudesolicitante
0,REGIAO DE SAUDE 10
1,REGIAO DE SAUDE 10


Com essas ações, quando pronto, o novo arquivo gerado após os tratamentos deve ser facilmente aberto em outros softwares.

### **3.4. Valores nulos**
Como visto em etapas anteriores, há valores nulos em certas colunas.<br>
Nessa seção, serão analisados tais valores com o objetivo de compreender se seguem um determinado padrão.

In [56]:
# Ajuste do DataFrame após a transformação dos caracteres especiais.
# Durante o tratamento anterior, os valores nulos foram substituídos pela string 'nan', o que pode mascarar valores nulos.

df.replace('nan', np.nan, inplace = True)

In [57]:
# Verificação dos valores nulos

df.isna().sum()

idsolicitacao                    0
situacaosolicitacao              0
data_solicitacao                 0
corregulador                     0
diasfilaespera                   0
complexidade                     0
especialidade                    0
especialidademae                 0
cid                              0
idpaciente                       0
municipio_residencia             0
idade                            0
cnes_solicitante                 0
unidadesolicitante               0
tipounidade                      0
municipiosolicitante             0
situacaoagenda              471913
datahora_consulta           471913
cnesexecutante              471913
unidadeexecutante           471913
municipio_executante        471913
statusregionalizacao        980232
centralregulacao                 0
centralregulacaoorigem           0
tipoconsulta                471913
coordenadoriasolicitante         0
regiaosaudesolicitante           0
possuiunidadeindicada            0
dtype: int64

Com os métodos isna() e sum(), verifica-se que:
- As colunas **situacaoagenda** , **datahora_consulta**, **cnesexecutante**, **unidadeexecutante**,  **municipio_executante**, **tipoconsulta** com mesma quantia de valores nulos.
- Já na coluna **statusregionalizacao**, aproximadamente 93.4% dos valores são nulos.

In [58]:
# Análise das colunas de mesmo comportamento
# Construção de novo DataFrame para essa análise, garantindo integridade nos dados trabalhados.

df_nulos = df[['situacaoagenda' , 'datahora_consulta', 'cnesexecutante', 'unidadeexecutante',  'municipio_executante', 'tipoconsulta']].copy()

In [59]:
df_nulos

,situacaoagenda,datahora_consulta,cnesexecutante,unidadeexecutante,municipio_executante,tipoconsulta
0,REALIZADA,2024-01-31,2237385.0,AMBULATORIO DE ESPECIALIDADES IAPI,PORTO ALEGRE,PRIMEIRA
1,REALIZADA,2024-03-19,2237385.0,AMBULATORIO DE ESPECIALIDADES IAPI,PORTO ALEGRE,PRIMEIRA
2,REALIZADA,2024-08-27,2237571.0,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,PORTO ALEGRE,PRIMEIRA
3,REALIZADA,2024-10-08,2237571.0,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,PORTO ALEGRE,PRIMEIRA
4,REALIZADA,2024-10-15,2237571.0,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,PORTO ALEGRE,PRIMEIRA
...,...,...,...,...,...,...
1048570,NaN,NaT,NaN,NaN,NaN,NaN
1048571,NaN,NaT,NaN,NaN,NaN,NaN
1048572,NaN,NaT,NaN,NaN,NaN,NaN
1048573,NaN,NaT,NaN,NaN,NaN,NaN


Observando o DataFrame **df_nulos**, há indícios que os dados são nulos ao mesmo tempo.<br>
Assim, removendo as linhas em que todas as colunas são nulas ao mesmo tempo, restará algum valor nulo? 

In [60]:
# Selecionando as linhas em que todos os valores são nulos ao mesmo tempo
mascara_todas_nan = df_nulos.isna().all(axis=1)

# Removendo as linhas onde todas as colunas são NaN
df_nulos_filtrado = df_nulos.drop(df_nulos[mascara_todas_nan].index)

In [61]:
# Conferindo a na estrutura do DataFrame

df_nulos_filtrado.shape

(576662, 6)

In [62]:
# Conferindo a quantidade de dados nulos restantes

df_nulos_filtrado.isna().sum()

situacaoagenda          0
datahora_consulta       0
cnesexecutante          0
unidadeexecutante       0
municipio_executante    0
tipoconsulta            0
dtype: int64

Considerando que o filtro **mascara_todas_nan** seleciona apenas as linhas em que todas as colunas são nulas ao mesmo tempo e que, após a limpeza não restaram dados nulos, é cofiável afirmar que tais colunas apresentam padrão de comportamento.<br>
Observado tal padrão, vale manter os registros, pois podem auxiliar em comparações futuras com as demais colunas.<br>
Pela natureza dos dados, não há padrão em cada coluna que indique substituição adequadas dos nulos.

Quanto a coluna **statusregionalizacao**, remover as linhas em que há valores nulos está fora de cogitação. 

Vale tratar com área de negócios:
- Preencher os valores nulos com expressão padrão, por exemplo: 'Sem registro'.
- Remoção da coluna **statusregionalizacao** devido pouca quantidade de dados preenchidos.

## **4. Load:** Carga dos dados transformados
Nessa etapa, o DataFrame com os dados tratados será inserido em um banco de dados MYSQL.

In [63]:
# Dados de conexão
usuario = 'root'
senha = '332654'
host = 'localhost'
banco_de_dados = 'projeto_saudepoa'

# Criando a string de conexão
string_conexao = f'mysql+mysqlconnector://{usuario}:{senha}@{host}/{banco_de_dados}'

# Criando a conexão
engine = create_engine(string_conexao)

In [64]:
nome_tabela = 'tbl_solicitacoes_2024_tratada' # Nome dado à tabela no destino

# Inserindo os dados no banco de dados
df.to_sql(nome_tabela, con=engine, if_exists='replace', index=False, chunksize=1000)

1048575

In [65]:
config = {
    'user': 'root',
    'password': '332654',
    'host': 'localhost',
    'database': 'projeto_saudepoa',
    'raise_on_warnings': True
}

conexao = mysql.connector.connect(**config)

cursor = conexao.cursor()
cursor.execute(f"ALTER TABLE {nome_tabela} ADD COLUMN id_registro INT AUTO_INCREMENT UNIQUE")
cursor.execute(f"ALTER TABLE {nome_tabela} MODIFY COLUMN id_registro INT AUTO_INCREMENT PRIMARY KEY FIRST")
cursor.close()
conexao.close()

In [66]:
# Confirmando a inserção dos dados

df_consulta = pd.read_sql(f'SELECT * FROM {nome_tabela}', con=engine)

df_consulta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 29 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   id_registro               1048575 non-null  int64         
 1   idsolicitacao             1048575 non-null  object        
 2   situacaosolicitacao       1048575 non-null  object        
 3   data_solicitacao          1048575 non-null  datetime64[ns]
 4   corregulador              1048575 non-null  object        
 5   diasfilaespera            1048575 non-null  int64         
 6   complexidade              1048575 non-null  object        
 7   especialidade             1048575 non-null  object        
 8   especialidademae          1048575 non-null  object        
 9   cid                       1048575 non-null  object        
 10  idpaciente                1048575 non-null  object        
 11  municipio_residencia      1048575 non-null  object

In [67]:
df_consulta.head(3)

,id_registro,idsolicitacao,situacaosolicitacao,data_solicitacao,corregulador,diasfilaespera,complexidade,especialidade,especialidademae,cid,...,cnesexecutante,unidadeexecutante,municipio_executante,statusregionalizacao,centralregulacao,centralregulacaoorigem,tipoconsulta,coordenadoriasolicitante,regiaosaudesolicitante,possuiunidadeindicada
0,1,5492110,REALIZADA,2024-04-10,LARANJA,2,ALTA,ONCOLOGIA PEDIATRICA,ONCOLOGIA,D360,...,2237571.0,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,PORTO ALEGRE,None,REGULACAO AMBULATORIAL,REGULACAO AMBULATORIAL,PRIMEIRA,13 CRS,REGIAO DE SAUDE 28,NAO
1,2,5071542,REALIZADA,2024-01-05,LARANJA,6,MEDIA,OFTALMOLOGIA GERAL ADULTO,OFTALMOLOGIA,H25,...,2259907.0,HOSPITAL SANTO ANGELO,SANTO ANGELO,None,REGULACAO REGIONAL 12CRS,REGULACAO REGIONAL 12CRS,PRIMEIRA,12 CRS,REGIAO DE SAUDE 11,SIM
2,3,6037225,REALIZADA,2024-08-21,AMARELO,22,MEDIA,CIRURGIA PLASTICA RECONSTRUCAO DE MAMA,CIRURGIA PLASTICA,C50,...,2265052.0,HOSPITAL FEMINA,PORTO ALEGRE,SEM_REGIONALIZACAO,CENTRAL DE REGULACAO AMBULATORIAL DO RIO GRAND...,CENTRAL DE REGULACAO AMBULATORIAL DO RIO GRAND...,PRIMEIRA,16 CRS,REGIAO DE SAUDE 29,NAO


## **5. Conclusões**
Sobre o objetivo da P1 do projeto:
- Os dados originais foram lidos com o encoder correto, garantindo a integridade das informações.
- As transformações realizadas adequaram os dados para carregamento em outros softwares de forma facilitada.
- Os dados foram inseridos com sucesso em um banco de dados MYSQL, atingindo o objetivo proposto.
<br>
<br>

Melhorias futuras:
- Inserção de dados: O documento original precisa ser disponibilizado manualmente para o código.
    - Automação com Web Scraping para busca automática dos arquivos no site.
- Revisão: O tempo de execução desse notebook é de aproximadamente 3min.
    - Refatoração do código, em especial na leitura inicial e na carga dos dados no MYSQL.
- Maior automação: Nesse notebook, há momentos de estudo do Dataset e dos tratamentos sugeridos, assim como há momentos de execução das transformações.
    - Construção de um script dedicado à execução das transformações para Datasets semelhantes.
- Segurança: A senha para conexão com o banco de dados está exposta no notebook.
    - Variável de ambiente para armazenar credenciais do banco de dados.